In [19]:
import pathlib, re, numpy as np, pandas as pd, plotly.graph_objects as go

In [20]:
# --- 1.  Explicit repo root ----------------------------------------------
ROOT_DIR = pathlib.Path(r"C:\Repositories\odi-data-visualization")

DATA_DIR  = ROOT_DIR / "data"
STYLE_NAME = "gauges_filled"
PLOTS_DIR  = ROOT_DIR / "plots" / STYLE_NAME

TEST_MODE  = False
TEST_DEPTS = ["Finance and Management", "Public Service"]   # ignored if TEST_MODE=False

# make sure output dir exists
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# ── helper to make every path/filename Windows-safe ───────────────────────
SAFE  = re.compile(r'[^0-9A-Za-z _()\-.]+')     # chars Windows allows
clean = lambda s, n=80: SAFE.sub("_", s)[:n].strip()

In [21]:
# ── 2.  Band colours & helper ─────────────────────────────────────────────
BANDS = [
    (1.00, 3.29, "#e53935"),   # red
    (3.30, 3.49, "#fb8c00"),   # orange
    (3.50, 3.79, "#fdd835"),   # yellow
    (3.80, 4.00, "#43a047"),   # green
]
def band_color(v, default="#999999"):
    for lo, hi, col in BANDS:
        if lo <= v <= hi:
            return col
    return default

In [22]:
# ── 3.  Extract first float from any cell ─────────────────────────────────
NUM_RE = re.compile(r"[-+]?\d*\.?\d+")
to_num = lambda s: float(NUM_RE.search(str(s)).group()) if NUM_RE.search(str(s)) else np.nan

In [23]:
# ── 4.  Load survey data → long format ────────────────────────────────────
def load_long():
    csvs = list((DATA_DIR).glob("*.csv"))
    if not csvs:
        raise FileNotFoundError(f"No CSVs in {DATA_DIR}")

    def tidy(p):
        df = pd.read_csv(p)
        if df.columns[0] != "Outcome":
            df = df.rename(columns={df.columns[0]:"Outcome"})
        return df[df["Outcome"].str.lower().str.strip() != "outcome"]

    wide = pd.concat([tidy(p) for p in csvs], ignore_index=True)
    wide["Outcome"] = wide["Outcome"].str.strip()
    wide.columns    = wide.columns.str.strip()
    order = list(dict.fromkeys(wide["Outcome"]))

    long = (wide
            .melt(id_vars="Outcome", var_name="Department", value_name="raw")
            .assign(Score=lambda d: d["raw"].apply(to_num))
            .dropna(subset=["Score"])
            .assign(Outcome=lambda d: pd.Categorical(d["Outcome"],
                                                     categories=order,
                                                     ordered=True)))
    return long[long["Score"].between(1.0, 4.0)]

long = load_long()

In [24]:
# ── 5.  Build filled-arc gauge ────────────────────────────────────────────
def build_gauge(score, title):
    fig = go.Figure(go.Indicator(
        mode   ="gauge+number",
        value  = score,
        title  = {"text": title, "font": {"size":18, "color":"#1f3d5a"}},
        number = {"font": {"size":38, "color":"#1f3d5a"}},
        gauge  = {
            "axis": {"range":[1,4],
                     "tickmode":"array",
                     "tickvals":np.linspace(1,4,7),
                     "tickfont":{"size":12}},
            "bar":  {"color": band_color(score), "thickness":0.45},
            "bgcolor":"#FFFFFF",
            "borderwidth":1,
            "bordercolor":"#444"
        }
    ))
    fig.update_layout(height=450, width=450,
                      margin=dict(t=60,b=20,l=20,r=20))
    return fig

In [25]:
# ── 6.  Save gauge into plots/<style>/<dept>/ ─────────────────────────────
def save_gauge(dept, outcome, score):
    dept_dir = PLOTS_DIR / clean(dept)          # keep this sanitised
    dept_dir.mkdir(parents=True, exist_ok=True)

    fig  = build_gauge(score, outcome)
    base = clean(outcome)                       # <<< use clean(), not replace()
    fig.write_html(dept_dir / f"{base}.html", include_plotlyjs="cdn")
    fig.write_image(dept_dir / f"{base}.png",  scale=2)
    print("   •", dept, "--", base + ".png")

In [26]:
# ── 7.  Generate gauges ───────────────────────────────────────────────────
targets = TEST_DEPTS if TEST_MODE else long["Department"].unique()

for dept in targets:
    sub = long[long["Department"].str.casefold() == dept.casefold()]
    if sub.empty:
        print(f"⚠️  No data for {dept}"); continue
    print("Generating", dept)
    for _, row in sub.iterrows():
        save_gauge(dept, row["Outcome"], row["Score"])

print("\nDone.  Plots saved under", PLOTS_DIR)

Generating Board of Health
   • Board of Health -- Scale Score.png
   • Board of Health -- I receive helpful ongoing feedback about my job performance..png
   • Board of Health -- I am rewarded for my performance..png
   • Board of Health -- My views and participation are valued..png
   • Board of Health -- Scale Score.png
   • Board of Health -- Leadership lets us know what City of Columbus is trying to accomplish..png
   • Board of Health -- The environment between leadership and staff is generally one of openness_ trust.png
   • Board of Health -- The leaders at City of Columbus are positive role models..png
   • Board of Health -- Scale Score.png
   • Board of Health -- City of Columbus_ policies and procedures_ including practices and work rules_ a.png
   • Board of Health -- Scale Score.png
   • Board of Health -- Employees in my department share responsibility for the success or failure of ou.png
   • Board of Health -- The distribution of work among the employees in my departme